https://www.theprojectspot.com/tutorial-post/ant-colony-optimization-for-hackers/10

In [1]:
import math
import random
from matplotlib import pyplot as plt
import import_ipynb
import Edg_Object
import time 
import threading 
import numpy as np



importing Jupyter notebook from Edg.ipynb


In [3]:
class Ant:
        class Sector:
            def __init__(self, vi_center, sector_key, nodes=[]):
                self.sector_key= sector_key
                self.vi_center= vi_center
                self.nodes = nodes  
        def __init__(self, alpha, beta, num_nodes, edges,ant_index=0, all_nodes=None):

            self.index= ant_index
            self.alpha = alpha
            self.beta = beta
            self.num_nodes = num_nodes
            self.all_nodes = all_nodes
            self.edges = edges
            self.tour = None
            self.distance = 0.0



        def _find_closest_node(self, vi, unvisited_nodes):
            if (vi== None) or (not unvisited_nodes): 
                return None
            min_closest_node= unvisited_nodes[0]

            for unvisited_node in unvisited_nodes: 


                if self.edges[vi][unvisited_node].weight < self.edges[vi][min_closest_node].weight:
                    min_closest_node= unvisited_node
            return min_closest_node

        def _create_sectors_for_vi(self, vi, unvisited_nodes, n_sectors):
            if (vi== None):
                return [] 
            list_of_sectors = [self.Sector(vi, sect, nodes=[]) for sect in range(n_sectors)]
            for unvisited_node in unvisited_nodes: 
                angle_in_180 = math.degrees(math.atan2((-1)*(self.all_nodes[unvisited_node][1] - self.all_nodes[vi][1]), self.all_nodes[unvisited_node][0] - self.all_nodes[vi][0])) # A = atan2 (Y - CenterY, X - CenterX)
                angle_in_360 = (angle_in_180+360)%360
                #starting_angle=360/n_secotos*current_sector # current_sector starts with 0 (0 -> n_secotos-1)
                #ending_angle=360/n_secotos*(current_sector+1) # current_sector starts with 0 (0 -> n_secotos-1)
 
                matching_sector=int(angle_in_360/(360/n_sectors))
                list_of_sectors[matching_sector].nodes.append(unvisited_node)

            return list_of_sectors


        def _treat_for_sectors(self, vi, nodes, n_sectors, nsize=10): #nsize= nodes in each cluster

            if (vi== None or len(nodes) < n_sectors):
                return -1
            unvisited_nodes=nodes
            cluster=1
            c_vi=[]
            clusters=[] #list of clusters (lists of nodes)
            list_of_sectors=self._create_sectors_for_vi(vi, unvisited_nodes, n_sectors)
            
            for sec in list_of_sectors:

                v_selected=self._find_closest_node(vi, sec.nodes)
                if v_selected != None:
                    c_vi.append(v_selected)
                    list_of_sectors[sec.sector_key].nodes.remove(v_selected)
                    unvisited_nodes.remove(v_selected)

            #print("now no sec")
            while unvisited_nodes:
                while len(c_vi) < nsize and unvisited_nodes:
                    v_selected=self._find_closest_node(vi, unvisited_nodes)
                    if v_selected != None:
                        c_vi.append(v_selected)
                        unvisited_nodes.remove(v_selected)


                clusters.append(c_vi.copy()) # deep copy

                c_vi.clear()

            return clusters 

            
                   
        def _create_clusters(self, n_size, n_sectors):
            all_clusters_lists=[]
            for vi in range(self.num_nodes):
                nodes = [node for node in range(self.num_nodes) if node != vi]   #בתוך המחלקה אנט אני צאיכה לעשות את זה כפונקציה getUnvisitedStates()
                all_clusters_lists.append(self._treat_for_sectors( vi, nodes, n_sectors, n_size))
                
            return all_clusters_lists



    

#-------------------------------

        def _calc_clusters_probabilities(self,all_clusters_lists, current_node): 
            denominator_eq4=0
            p_clusters=[]
            for cluster in all_clusters_lists[current_node]:
                sum_of_nodes_eta=0 #eta is 1/distace(current_node->next optional node)
                sum_of_nodes_tau=0 #tau is 1/distace(current_node->next optional node)

                for node in cluster:  
                    sum_of_nodes_eta+=(1/self.edges[current_node][node].weight)
                    sum_of_nodes_tau+=self.edges[current_node][node].pheromone
                eta_current_cluster_eq5= len(cluster)*sum_of_nodes_eta 
                tau_current_cluster_eq6= (1/len(cluster))*sum_of_nodes_tau
                numerator_eq4 = eta_current_cluster_eq5*tau_current_cluster_eq6
                denominator_eq4+= numerator_eq4

                p_clusters.append(numerator_eq4)

            probabilities_of_next_clusters= [x/denominator_eq4 for x in p_clusters]
 
            return probabilities_of_next_clusters

        def _calc_nodes_probabilities(self,candidates_nodes_list, current_node): 
            denominator_eq1=0
            p_nodes=[]
            sum_of_nodes_eta=0 #eta is 1/distace(current_node->next optional node)
            sum_of_nodes_tau=0 #tau is 1/distace(current_node->next optional node)
            for node in candidates_nodes_list:
                node_eta=(1/self.edges[current_node][node].weight) 
                nodes_tau=self.edges[current_node][node].pheromone
                numerator_eq1 = node_eta*nodes_tau
                denominator_eq1+= numerator_eq1
            
                p_nodes.append(numerator_eq1)
            probabilities_of_next_nodes= [x/denominator_eq1 for x in p_nodes] 
   
            return probabilities_of_next_nodes


        def _roulette_wheel(self, probabilities_of_next_items):
            random_value = random.uniform(0.0, 1.0)
            wheel_position=0
            for item_probability in probabilities_of_next_items:
                wheel_position +=item_probability
                if wheel_position >= random_value:
                    return probabilities_of_next_items.index(item_probability)
            return 0 


        def _select_node2(self, current_node , all_clusters_lists, probabilities_of_next_clusters ): 
            
            first=1

            selected_cluster_index= self._roulette_wheel(probabilities_of_next_clusters)
            selected_cluster= all_clusters_lists[current_node][selected_cluster_index]
            unvisited_nodes = [node for node in range(self.num_nodes) if node not in self.tour and node in selected_cluster] 

            while not unvisited_nodes: 
                if first==1: 
                    selected_cluster_index=0 
                    first=0
                else: selected_cluster_index+=1 
                selected_cluster= all_clusters_lists[current_node][selected_cluster_index]
                unvisited_nodes = [node for node in range(self.num_nodes) if node not in self.tour and node in selected_cluster]  

            probabilities_of_next_node=self._calc_nodes_probabilities(unvisited_nodes, current_node)
            selected_node_index= self._roulette_wheel(probabilities_of_next_node)
            selected_node=unvisited_nodes[selected_node_index]

            return selected_node
            

        def find_tour(self):
            all_clusters_lists=self._create_clusters(7,6)
            self.tour = [random.randint(0, self.num_nodes - 1)] 
            current_node=self.tour[-1]
            probabilities_of_next_clusters=self._calc_clusters_probabilities(all_clusters_lists, current_node)

            while len(self.tour) < self.num_nodes: 
                current_node=self._select_node2(current_node, all_clusters_lists, probabilities_of_next_clusters )
                self.tour.append(current_node)
            return self.tour

        def get_distance(self):
            self.distance = 0.0
            for i in range(self.num_nodes):
                self.distance += self.edges[self.tour[i]][self.tour[(i + 1) % self.num_nodes]].weight
            return self.distance



